In [3]:
import os
import random
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, classification_report
from sklearn.utils.class_weight import compute_class_weight

import shap
import optuna

import torch
import torch.nn as nn
from pytorch_tabnet.pretraining import TabNetPretrainer
from pytorch_tabnet.tab_model import TabNetClassifier
from typing import Tuple, List

Some useful global constants and setting the seed

In [4]:
SEED = 7
CORR_THRESHOLD = 0.85 # correlation threshold for dimensionality reduction
TEST_SIZE = 0.30 # train-test split
VAL_SIZE = 0.20  # train-val split   
N_TRIALS = 50                     
MAX_PRETRAIN_EPOCHS = 150
MAX_FINETUNE_EPOCHS = 200
EARLY_STOPPING_PATIENCE = 30
BATCH_SIZE = 2048
VIRTUAL_BATCH_SIZE = 256
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

FOCAL_ALPHA = 0.75                 # Higher alpha for rare positive class
FOCAL_GAMMA = 2.0   

In [5]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Soem useful helper functions

In [ ]:
def bootstrap_auc_ci(y_true, y_scores, n_bootstraps=2000, ci=0.95):
    """ 
    Simple Bootstrapping method to get an confidence interval on the AUROC score.
    """
    rng = np.random.default_rng(42)
    aucs = []

    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    for _ in range(n_bootstraps):
        idx = rng.integers(0, len(y_true), len(y_true))
        if len(np.unique(y_true[idx])) < 2:
            continue
        aucs.append(roc_auc_score(y_true[idx], y_scores[idx]))

    lower = np.percentile(aucs, (1 - ci) / 2 * 100)
    upper = np.percentile(aucs, (1 + ci) / 2 * 100)
    return np.mean(aucs), lower, upper

## Loading the dataset, pre-processing, and analysing the data

In [7]:
cohort_data = pd.read_csv('../cohort_data_new.csv')
cohort_data

,icustay_id,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,...,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,gender,icu_los_hours,target
0,200003,13.375000,3.583195,9.0,21.0,25.250000,3.105295,18.0,28.0,7.771429,...,10.0,21.0,26.471429,13.176711,13.2,43.9,48,M,141,0
1,200007,15.500000,2.121320,14.0,17.0,23.000000,1.414214,22.0,24.0,8.900000,...,8.0,10.0,10.300000,1.272792,9.4,11.2,44,M,30,0
2,200009,9.500000,2.121320,8.0,11.0,23.333333,2.081666,21.0,25.0,8.000000,...,15.0,21.0,12.471429,1.471637,10.5,14.3,47,F,51,0
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.900000,NaN,4.9,4.9,33,F,10,0
4,200014,10.000000,1.732051,9.0,12.0,24.000000,1.000000,23.0,25.0,7.733333,...,21.0,24.0,13.233333,2.203028,10.7,14.7,85,M,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,299992,15.375000,2.856153,11.0,25.0,23.125000,2.609556,15.0,26.0,8.307143,...,8.0,23.0,14.134783,3.781727,8.1,22.1,41,M,499,0
30485,299993,9.400000,1.341641,8.0,11.0,29.600000,2.073644,26.0,31.0,8.000000,...,12.0,15.0,12.600000,0.605530,12.0,13.3,26,M,67,0
30486,299994,16.157895,2.477973,13.0,24.0,21.631579,3.451417,17.0,31.0,8.100000,...,28.0,63.0,10.076190,2.642329,5.3,14.5,74,F,152,1
30487,299998,11.500000,1.732051,10.0,14.0,23.500000,1.290994,22.0,25.0,8.800000,...,20.0,22.0,9.900000,1.210372,7.9,11.0,87,M,46,1


In [8]:
print(f"Dataset shape: {cohort_data.shape}")
print(f"Readmission rate: {cohort_data['target'].mean() * 100:.2f}%")

Dataset shape: (30489, 93)
Readmission rate: 10.74%


In [9]:
lab_cols = [
    'anion_gap_mean', 'anion_gap_min', 'anion_gap_max', 'anion_gap_sd',
    'bicarbonate_mean', 'bicarbonate_min', 'bicarbonate_max', 'bicarbonate_sd',
    'calcium_total_mean', 'calcium_total_min', 'calcium_total_max', 'calcium_total_sd',
    'chloride_mean', 'chloride_min', 'chloride_max', 'chloride_sd',
    'creatinine_mean', 'creatinine_min', 'creatinine_max', 'creatinine_sd',
    'glucose_mean', 'glucose_min', 'glucose_max', 'glucose_sd',
    'hematocrit_mean', 'hematocrit_min', 'hematocrit_max', 'hematocrit_sd',
    'hemoglobin_mean', 'hemoglobin_min', 'hemoglobin_max', 'hemoglobin_sd',
    'mchc_mean', 'mchc_min', 'mchc_max', 'mchc_sd',
    'mcv_mean', 'mcv_min', 'mcv_max', 'mcv_sd',
    'magnesium_mean', 'magnesium_min', 'magnesium_max', 'magnesium_sd',
    'pt_mean', 'pt_min', 'pt_max', 'pt_sd',
    'phosphate_mean', 'phosphate_min', 'phosphate_max', 'phosphate_sd',
    'platelet_count_mean', 'platelet_count_min', 'platelet_count_max', 'platelet_count_sd',
    'potassium_mean', 'potassium_min', 'potassium_max', 'potassium_sd',
    'rdw_mean', 'rdw_min', 'rdw_max', 'rdw_sd',
    'red_blood_cells_mean', 'red_blood_cells_min', 'red_blood_cells_max', 'red_blood_cells_sd',
    'sodium_mean', 'sodium_min', 'sodium_max', 'sodium_sd',
    'urea_nitrogen_mean', 'urea_nitrogen_min', 'urea_nitrogen_max', 'urea_nitrogen_sd',
    'white_blood_cells_mean', 'white_blood_cells_min', 'white_blood_cells_max', 'white_blood_cells_sd',
    'age', 'icu_los_hours'
]

REmove the ICUstay_id and the gender

In [10]:
drop_cols = [c for c in cohort_data.columns if 'icustay_id' in c.lower() or 'gender' in c.lower()]
df = cohort_data.drop(columns=['icustay_id', 'gender'], errors='ignore')

X = df.drop(columns=['target'])
y = df['target']

Trying out some feature engineering

In [ ]:
# X = X.select_dtypes(include=['number']).replace([np.inf, -np.inf], np.nan)
# print(f"initial feature matrix shape: {X.shape}")

# def create_engineered_features(df: pd.DataFrame) -> pd.DataFrame:
#     df_eng = df.copy()
    
#     # BUN/Creatinine ratio (kidney function indicator)
#     if 'urea_nitrogen_mean' in df_eng.columns and 'creatinine_mean' in df_eng.columns:
#         df_eng['bun_creatinine_ratio'] = (
#             df_eng['urea_nitrogen_mean'] / (df_eng['creatinine_mean'] + 1e-6)
#         )

#     # Variability indices (physiological instability)
#     variability_features = []
#     for base_name in ['glucose', 'potassium', 'sodium', 'hemoglobin']:
#         mean_col = f'{base_name}_mean'
#         sd_col = f'{base_name}_sd'
#         if mean_col in df_eng.columns and sd_col in df_eng.columns:
#             cv_col = f'{base_name}_cv'
#             df_eng[cv_col] = df_eng[sd_col] / (df_eng[mean_col] + 1e-6)
#             variability_features.append(cv_col)
    
#     # Range features (max - min)
#     for base_name in ['glucose', 'creatinine', 'potassium']:
#         max_col = f'{base_name}_max'
#         min_col = f'{base_name}_min'
#         if max_col in df_eng.columns and min_col in df_eng.columns:
#             range_col = f'{base_name}_range'
#             df_eng[range_col] = df_eng[max_col] - df_eng[min_col]
    
#     return df_eng

# X_engineered = create_engineered_features(X)
# print(f"final feature matrix shape: {X_engineered.shape}")

initial feature matrix shape: (30489, 90)
final feature matrix shape: (30489, 98)


Dimensionality reduction by deleting the columns with high correlation.

In [12]:
# corr = X_engineered.corr().abs()
# upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
# to_drop = [col for col in upper.columns if any(upper[col] >= CORR_THRESHOLD)]

# print(f"dropping {len(to_drop)} features")
# X_reduced = X_engineered.drop(columns=to_drop, errors='ignore')
# print(f"final feature count: {X_reduced.shape[1]}")
# feature_names = X_reduced.columns.tolist()

Creating the final train-val-test sets

In [13]:
# separate test set
X_temp, X_test, y_temp, y_test = train_test_split(X, y.values, test_size=TEST_SIZE, random_state=SEED, stratify=y.values)

# separate validation set 
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=VAL_SIZE, random_state=SEED, stratify=y_temp)

print(f"Train set: {X_train.shape[0]} samples ({y_train.mean()*100:.2f}% readmission)")
print(f"Validation set: {X_val.shape[0]} samples ({y_val.mean()*100:.2f}% readmission)")
print(f"Test set: {X_test.shape[0]} samples ({y_test.mean()*100:.2f}% readmission)")

# SimpleImputation using median strat and scaling;
# Imputation - FIT on train only and avoidning data leakage:
imputer = SimpleImputer(strategy="median")
X_train_imputed = imputer.fit_transform(X_train)
X_val_imputed = imputer.transform(X_val)
X_test_imputed = imputer.transform(X_test)

# Scaling - FIT on train only
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_imputed)
X_val_scaled = scaler.transform(X_val_imputed)
X_test_scaled = scaler.transform(X_test_imputed)

Train set: 17073 samples (10.74% readmission)
Validation set: 4269 samples (10.75% readmission)
Test set: 9147 samples (10.75% readmission)


### Pretraining the TABNet

In [14]:
def run_pretraining(X_train: np.ndarray, X_val: np.ndarray, pretrain_params: dict):
    """Run unsupervised pretraining"""
    pretrainer = TabNetPretrainer(**pretrain_params)
    pretrainer.fit(
        X_train=X_train,
        eval_set=[X_val],
        max_epochs=MAX_PRETRAIN_EPOCHS,
        patience=EARLY_STOPPING_PATIENCE,
        batch_size=BATCH_SIZE,
        virtual_batch_size=VIRTUAL_BATCH_SIZE,
        num_workers=0,
        drop_last=False
    )
    
    print("Pretraining complete!")
    return pretrainer

pretrain_params = dict(
    n_d=32, 
    n_a=32,
    n_steps=5,
    gamma=1.5,
    optimizer_fn=torch.optim.Adam,
    optimizer_params=dict(lr=1e-3),
    mask_type="entmax",
    device_name=DEVICE
)

pretrainer = run_pretraining(X_train_scaled, X_val_scaled, pretrain_params)

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 12.34471| val_0_unsup_loss_numpy: 3.271250009536743|  0:00:03s
epoch 1  | loss: 11.24094| val_0_unsup_loss_numpy: 3.1070899963378906|  0:00:06s
epoch 2  | loss: 9.99972 | val_0_unsup_loss_numpy: 3.025629997253418|  0:00:09s
epoch 3  | loss: 9.19899 | val_0_unsup_loss_numpy: 3.0878798961639404|  0:00:12s
epoch 4  | loss: 8.46691 | val_0_unsup_loss_numpy: 3.0285000801086426|  0:00:14s
epoch 5  | loss: 7.69079 | val_0_unsup_loss_numpy: 3.1906800270080566|  0:00:17s
epoch 6  | loss: 7.14098 | val_0_unsup_loss_numpy: 3.362839937210083|  0:00:20s
epoch 7  | loss: 6.58374 | val_0_unsup_loss_numpy: 2.852479934692383|  0:00:23s
epoch 8  | loss: 6.17838 | val_0_unsup_loss_numpy: 2.8965299129486084|  0:00:25s
epoch 9  | loss: 5.66219 | val_0_unsup_loss_numpy: 2.9274098873138428|  0:00:28s
epoch 10 | loss: 5.28782 | val_0_unsup_loss_numpy: 2.619230031967163|  0:00:32s
epoch 11 | loss: 4.74309 | val_0_unsup_loss_numpy: 2.4046199321746826|  0:00:36s
epoch 12 | loss: 4.38431 | val_0_

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Hyperparam Search

In [20]:
def make_objective(X_train, y_train, X_val, y_val, pretrainer):
    """Create Optuna objective function"""
    
    def objective(trial):
        # Hyperparameters to tune
        n_d = trial.suggest_int("n_d", 16, 64)
        n_a = trial.suggest_int("n_a", 16, 64)
        n_steps = trial.suggest_int("n_steps", 3, 7)
        gamma = trial.suggest_float("gamma", 1.0, 2.5)
        lambda_sparse = trial.suggest_float("lambda_sparse", 1e-6, 1e-3, log=True)
        lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
        mask_type = trial.suggest_categorical("mask_type", ["sparsemax", "entmax"])
        
        clf = TabNetClassifier(n_d=n_d, n_a=n_a, n_steps=n_steps, gamma=gamma, lambda_sparse=lambda_sparse, optimizer_fn=torch.optim.Adam,optimizer_params=dict(lr=lr), mask_type=mask_type, device_name=DEVICE, verbose=0)
        
        try:
            clf.fit(
                X_train, y_train,
                eval_set=[(X_val, y_val)],
                eval_name=["val"],
                eval_metric=["auc"],
                max_epochs=MAX_FINETUNE_EPOCHS,
                patience=20, 
                batch_size=BATCH_SIZE,
                virtual_batch_size=VIRTUAL_BATCH_SIZE,
                num_workers=0,
                drop_last=False,
                from_unsupervised=pretrainer,
            )
        except Exception as e:
            print(f"Trial failed: {e}")
            raise optuna.exceptions.TrialPruned()
        
        pred_proba = clf.predict_proba(X_val)[:, 1]
        auc = roc_auc_score(y_val, pred_proba)
        
        return auc
    
    return objective

In [21]:
print("hyperparameter optimization under way")
study = optuna.create_study(direction="maximize", study_name="tabnet_readmission_10pct")

objective = make_objective(X_train_scaled, y_train, X_val_scaled, y_val, pretrainer)
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)
print(f"\nBest trial: {study.best_trial.number}")
print(f"Best validation AUROC: {study.best_value:.4f}")
print("\nBest hyperparameters:")
for key, value in study.best_trial.params.items():
    print(f"  {key}: {value}")

best_params = study.best_trial.params

[I 2025-11-30 08:14:38,931] A new study created in memory with name: tabnet_readmission_10pct


hyperparameter optimization under way


  0%|          | 0/50 [00:00<?, ?it/s]

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from sparsemax to entmax
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_a changed from 16 to 32
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_d changed from 18 to 32
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_steps changed from 3 to 5
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 98 with best_epoch = 78 and best_val_auc = 0.60442


...

[I 2025-11-30 12:41:06,688] Trial 48 finished with value: 0.6892365578485695 and parameters: {'n_d': 64, 'n_a': 45, 'n_steps': 5, 'gamma': 1.0025942828427767, 'lambda_sparse': 0.0004295411263328813, 'lr': 0.002272200268295512, 'mask_type': 'sparsemax'}. Best is trial 25 with value: 0.6913814694731787.


c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: mask_type changed from sparsemax to entmax
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_a changed from 45 to 32
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_d changed from 58 to 32
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")



Early stopping occurred at epoch 61 with best_epoch = 41 and best_val_auc = 0.66886


c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


[I 2025-11-30 12:44:13,476] Trial 49 finished with value: 0.6688601833267573 and parameters: {'n_d': 58, 'n_a': 45, 'n_steps': 5, 'gamma': 1.367113271007407, 'lambda_sparse': 0.0004104048356351314, 'lr': 0.004180368268552419, 'mask_type': 'sparsemax'}. Best is trial 25 with value: 0.6913814694731787.

Best trial: 25
Best validation AUROC: 0.6914

Best hyperparameters:
  n_d: 59
  n_a: 57
  n_steps: 5
  gamma: 1.7111039914098805
  lambda_sparse: 0.00024137154945623953
  lr: 0.0035177974693979767
  mask_type: entmax


Retraining using the best params

In [22]:
final_model = TabNetClassifier(n_d=best_params["n_d"], n_a=best_params["n_a"], n_steps=best_params["n_steps"], gamma=best_params["gamma"], lambda_sparse=best_params["lambda_sparse"], optimizer_fn=torch.optim.Adam, optimizer_params=dict(lr=best_params["lr"]), mask_type=best_params.get("mask_type", "entmax"), device_name=DEVICE, verbose=1)

final_model.fit(X_train_scaled, y_train, eval_set=[(X_val_scaled, y_val)], eval_name=["val"], eval_metric=["auc"], max_epochs=MAX_FINETUNE_EPOCHS, patience=EARLY_STOPPING_PATIENCE, batch_size=BATCH_SIZE, virtual_batch_size=VIRTUAL_BATCH_SIZE, num_workers=0, drop_last=False, from_unsupervised=pretrainer, weights=1)
final_model.save_model("tabnet_readmission_final")
print("Model saved.")

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_a changed from 57 to 32
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:118: UserWarning: Pretraining: n_d changed from 59 to 32
  warnings.warn(wrn_msg)
c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\abstract_model.py:248: UserWarning: Loading weights from unsupervised pretraining
  warnings.warn("Loading weights from unsupervised pretraining")


epoch 0  | loss: 1.13584 | val_auc: 0.52226 |  0:00:02s
epoch 1  | loss: 0.89311 | val_auc: 0.55845 |  0:00:04s
epoch 2  | loss: 0.82872 | val_auc: 0.5585  |  0:00:06s
epoch 3  | loss: 0.78105 | val_auc: 0.57096 |  0:00:08s
epoch 4  | loss: 0.76253 | val_auc: 0.59085 |  0:00:10s
epoch 5  | loss: 0.73844 | val_auc: 0.60154 |  0:00:13s
epoch 6  | loss: 0.72085 | val_auc: 0.60592 |  0:00:15s
epoch 7  | loss: 0.71155 | val_auc: 0.62129 |  0:00:17s
epoch 8  | loss: 0.70354 | val_auc: 0.61798 |  0:00:19s
epoch 9  | loss: 0.69408 | val_auc: 0.61406 |  0:00:21s
epoch 10 | loss: 0.69128 | val_auc: 0.61359 |  0:00:23s
epoch 11 | loss: 0.69554 | val_auc: 0.62069 |  0:00:25s
epoch 12 | loss: 0.69249 | val_auc: 0.62463 |  0:00:28s
epoch 13 | loss: 0.6868  | val_auc: 0.62261 |  0:00:30s
epoch 14 | loss: 0.67813 | val_auc: 0.63902 |  0:00:32s
epoch 15 | loss: 0.68438 | val_auc: 0.63961 |  0:00:34s
epoch 16 | loss: 0.67382 | val_auc: 0.63584 |  0:00:36s
epoch 17 | loss: 0.67438 | val_auc: 0.64296 |  0

c:\Users\risha\miniconda3\envs\ML_Torch\Lib\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Successfully saved model at tabnet_readmission_final.zip
Model saved.


Evaluation

In [23]:
y_test_proba = final_model.predict_proba(X_test_scaled)[:, 1]

y_test_pred_default = (y_test_proba >= 0.5).astype(int)

auc_score = roc_auc_score(y_test, y_test_proba)
auc_mean, auc_lower, auc_upper = bootstrap_auc_ci(y_test, y_test_proba, n_bootstraps=2000)

print(f"\nAUROC: {auc_score:.4f}")
print(f"95% CI: [{auc_lower:.4f}, {auc_upper:.4f}]")

# Classification report (default threshold)
print("Classification Report (threshold=0.5)")
print(classification_report(y_test, y_test_pred_default, target_names=['No Readmission', 'Readmission']))


AUROC: 0.6507
95% CI: [0.6315, 0.6684]
Classification Report (threshold=0.5)
                precision    recall  f1-score   support

No Readmission       0.93      0.62      0.75      8164
   Readmission       0.17      0.62      0.26       983

      accuracy                           0.62      9147
     macro avg       0.55      0.62      0.50      9147
  weighted avg       0.85      0.62      0.69      9147



Plotting

In [24]:
# Get feature importance from TabNet's attention mechanism
feature_importance = final_model.feature_importances_

# Create dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\nTop 20 most important features:")
print(importance_df.head(20).to_string(index=False))

NameError: name 'feature_names' is not defined